# Large-scale partitioned circuit
Simulate a large circuit that triggers partitioning in QuASAr and record execution metrics.

## Expected backend transition
The circuit combines a stabilizer GHZ block with a non-Clifford QFT block. The planner should first use the `TABLEAU` backend and then transition to the `DECISION_DIAGRAM` backend.

In [ ]:
import json, pathlib
import pandas as pd
from quasar import Backend, NoFeasibleBackendError
from quasar.circuit import Circuit, Gate
from quasar.simulation_engine import SimulationEngine
from quasar.analyzer import CircuitAnalyzer
from benchmarks.circuits import ghz_circuit, qft_circuit

def large_partitioned_circuit(n: int) -> Circuit:
    half = n // 2
    ghz = ghz_circuit(half, use_classical_simplification=False)
    qft = qft_circuit(half, use_classical_simplification=False)
    gates = list(ghz.gates)
    gates += [Gate(g.gate, [q + half for q in g.qubits], g.params) for g in qft.gates]
    gates += [
        Gate('CX', [0, half]),
        Gate('CX', [half - 1, n - 1]),
    ]
    return Circuit(gates, use_classical_simplification=False)

n_qubits = 16
circuit = large_partitioned_circuit(n_qubits)
engine = SimulationEngine()
analyzer = CircuitAnalyzer(circuit, estimator=engine.planner.estimator)
analysis = analyzer.analyze()
results = None
try:
    plan = engine.scheduler.prepare_run(circuit, analysis=analysis)
    partitions = circuit.ssd.partitions
    assert len(partitions) == 2
    assert [p.backend for p in partitions] == [Backend.TABLEAU, Backend.DECISION_DIAGRAM]
    print([(p.backend.name, p.qubits) for p in partitions])
    _, metrics = engine.scheduler.run(circuit, plan, analysis=analysis, instrument=True)
    results = pd.DataFrame([
        {
            'runtime_s': metrics.cost.time,
            'peak_memory_bytes': metrics.cost.memory,
            'backend_switches': metrics.backend_switches,
            'conversions': len(metrics.conversion_durations),
        }
    ])
except NoFeasibleBackendError as exc:
    print(f'No feasible backend: {exc}')
results


In [ ]:
import json, pathlib
try:
    import ipynbname
    nb_name = ipynbname.path().stem
except Exception:  # pragma: no cover
    nb_name = 'large_scale_partitioning'
params = {'qubits': n_qubits}
pathlib.Path('../results').mkdir(exist_ok=True)
with open(f'../results/{nb_name}_params.json', 'w') as f:
    json.dump(params, f, indent=2, default=str)
if results is not None:
    with open(f'../results/{nb_name}_results.json', 'w') as f:
        json.dump(results.to_dict(orient='records'), f, indent=2, default=str)
    print(json.dumps(params, indent=2, default=str))
else:
    print('Planning failed; no metrics recorded.')
